Делаем прогноз слов рекуррентной сетью. Embedding слой | #21 нейросети на Python

In [2]:

import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [4]:
with open('train_data_false', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [22]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [27]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('я', 19), ('притягиваю', 1), ('только', 18), ('плохое', 1), ('кому', 2), ('нужен', 2), ('с', 4), ('такой', 5), ('внешностью', 2), ('не', 55)]


In [28]:
data = tokenizer.texts_to_sequences([texts])
# res = to_categorical(data[0], num_classes=maxWordsCount)
# print(res.shape)
res = np.array( data[0] )

inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [29]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0058 - loss: 6.9034 
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1313 - loss: 6.6310
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.0619 - loss: 5.5922
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0918 - loss: 5.2564
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.0963 - loss: 5.0067
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0977 - loss: 4.7713
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1129 - loss: 4.5619
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1636 - loss: 4.1978
Epoch 9/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.2844 - loss: 3.7395
Epoch 10/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.4301 - loss: 3.3923
Epoch 11/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5503 - loss: 2.9259
Epoch 12/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy

In [38]:
from keras.preprocessing.sequence import pad_sequences

def buildPhrase(texts, str_len=20, inp_words=3, maxWordsCount=1000):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]

    # Если последовательность слишком короткая, добавляем padding
    if len(data) < inp_words:
        data = pad_sequences([data], maxlen=inp_words, padding='pre')[0].tolist()

    for _ in range(str_len):
        # Берем последние inp_words индексов для входа
        x = data[-inp_words:]
        inp = np.expand_dims(x, axis=0)  # Добавляем измерение для batch size

        # Предсказание модели
        pred = model.predict(inp, verbose=0)
        indx = pred.argmax(axis=1)[0]

        # Добавляем предсказанный индекс в последовательность
        data.append(indx)

        # Преобразуем индекс в слово и добавляем к результату
        res += " " + tokenizer.index_word.get(indx, "")

    return res



res = buildPhrase("абсолютно", str_len=20, inp_words=3, maxWordsCount=1000)
print(res)

абсолютно не уйдешь все не повезло никогда в бедной не убежит на делай все не последний никогда мою начальников не хронических
